## Hands on Practical Deep learning with PyTorch
### Sign Language Digits Dataset
https://www.kaggle.com/ankitjha/hands-on-practical-deep-learning-with-pytorch/notebook

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from sklearn.model_selection import train_test_split

In [18]:
data = np.load('./data/X.npy')*255
target = np.load('./data/Y.npy')
Y = np.zeros(data.shape[0])
Y[:204] = 9
Y[204:409] = 0
Y[409:615] = 7
Y[615:822] = 6
Y[822:1028] = 1
Y[1028:1236] = 8
Y[1236:1443] = 4
Y[1443:1649] = 3
Y[1649:1855] = 2
Y[1855:] = 5
X_train, X_test, y_train, y_test = train_test_split(data, Y, test_size = .02, random_state = 2) 
## splitting into train and test set

In [19]:
X_train.shape, X_train.max(), X_train.min()

((2020, 64, 64), 255.0, 1.0)

-  Data must be wrapped on a Dataset parent class where the methods getitem and len must be overrided. Not that, the data is not loaded on memory by now.
- The Dataloader reads the data and puts it into memory.

- torchvision - It is used to load and prepare dataset. Using it you can create transformations on the input data. 

- transforms - It is used for preprocessing images and performing operations sequentially. 

- num_workers - It is used for multiprocessing.Normally, num_workers = 4 * (number of gpus) works well.

In [20]:
print(X_train.shape)
x = X_train[77:78,...]
print(x.shape)
x = x.reshape((-1, 64, 64)).astype(np.uint8)[:, :, :, None]
print(x.shape)

(2020, 64, 64)
(1, 64, 64)
(1, 64, 64, 1)


In [9]:
from torch.utils.data import Dataset


class DatasetProcessing(Dataset):
    def __init__(self, data, target, transform=None):
        # used to initialise the class variables - transform, data, target
        self.transform = transform
        # print(data.shape)
        self.data = torch.from_numpy(data.reshape((-1, 64, 64))[:, None, :, :])
        # print(self.data.shape)
        self.target = torch.from_numpy(target).long()
        # needs to be in torch.LongTensor dtype

    def __getitem__(self, index):
        # used to retrieve the X and y index value and return it
        return self.transform(self.data[index]), self.target[index]
#         return self.data[index], self.target[index]

    def __len__(self):
        # returns the length of the data
        return len(list(self.data))

In [10]:
import torch
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Normalize([0.5], [0.5])
])
dset_train = DatasetProcessing(X_train, y_train, transform)
train_loader = torch.utils.data.DataLoader(
    dset_train, batch_size=2020, shuffle=True, num_workers=4)

In [12]:
# print(X_train.shape, y_train.shape)
print(
      np.min(X_train), 
      np.max(X_train),
#       np.min(y_train),
#       np.max(y_train),
     )
xx, yy = next(iter(train_loader))
print(xx.shape, yy.shape)
print(
      torch.min(xx),
      torch.max(xx),
#       torch.min(yy), 
#       torch.max(yy),
     )

0.003921569 1.0
torch.Size([2020, 1, 64, 64]) torch.Size([2020])
-0.9921568632125854 1.0


In [13]:
data= torch.from_numpy(X_train.reshape((-1, 64, 64))[:, None, :, :])
data.shape, data[1].shape

(torch.Size([2020, 1, 64, 64]), torch.Size([1, 64, 64]))

In [15]:
data[1]*255


(0 ,.,.) = 
  122  125  130  ...   115  114  111
  124  128  132  ...   116  114  112
  125  129  134  ...   117  115  112
      ...         ⋱        ...      
  115  117  120  ...   112  109  107
  113  116  118  ...   111  109  107
  110  113  115  ...   110  108  105
[torch.FloatTensor of size 1x64x64]

In [ ]:
results = transforms.ToPILImage()(X_train.reshape((-1, 64, 64)).astype(np.uint8)[:, :, :, None][1])
type(results)

In [ ]:
from torch import nn
from torch.autograd import Variable
print(xx.shape)
nn.Sequential(nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1))(Variable(xx)).shape

In [ ]:
dset_test = DatasetProcessing(X_test, y_test, transform)
test_loader = torch.utils.data.DataLoader(
    dset_test, batch_size=4, shuffle=True, num_workers=0)

In [ ]:
import matplotlib.pyplot as plt
for num, x in enumerate(X_train[0:6]):
    plt.subplot(1,6,num+1)
    plt.axis('off')
    plt.imshow(x)
    plt.title(y_train[num])

In [ ]:
from torch import nn
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(32 * 32 * 32, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)

        return x

In [ ]:
import torch.optim as optim
from torch.optim import lr_scheduler

model = Net()

optimizer = optim.Adam(model.parameters(), lr=0.01)

criterion = nn.CrossEntropyLoss()

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

if torch.cuda.is_available():
    print('Cuda!')
    model = model.cuda()
    criterion = criterion.cuda()

In [ ]:
from torch.autograd import Variable
def train(epoch):
    model.train()
    exp_lr_scheduler.step()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        #print(data.size())
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        #print(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)

        loss.backward()
        optimizer.step()

        if (batch_idx + 1) % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_idx + 1) * len(data), len(train_loader.dataset),
                100. * (batch_idx + 1) / len(train_loader), loss.data[0]))

In [ ]:
import torch.nn.functional as F
def evaluate(data_loader):
    model.eval()
    loss = 0
    correct = 0

    for data, target in data_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()

        output = model(data)
        loss += F.cross_entropy(output, target, size_average=False).data[0]
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    loss /= len(data_loader.dataset)

    print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))

In [ ]:
n_epochs = 2

for epoch in range(n_epochs):
    train(epoch)
    evaluate(train_loader)